In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F

import torch
import torch.nn as nn
import torchmetrics

from tqdm import tqdm 
import hydra
# from omegaconf import DictConfig

from clearml import Task, Logger

from datamodule import data, utils
from models import LSTM, CNN, Bertweet, Roberta

from transformers import BertModel
bert = BertModel.from_pretrained('bert-base-uncased')

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
DEVICE = device

In [ ]:
from transformers import RobertaModel, RobertaConfig, RobertaTokenizer

roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
tokens_pt = roberta_tokenizer("in desperate need of and i can not stress this enough spring break", padding='max_length', max_length=12, truncation=True, return_tensors='pt')  
tokens_pt

In [60]:

class BERTSentiment(nn.Module):
    def __init__(self,
                 num_layers,
                 bidirectional,
                 output_dim):
        super(BERTSentiment, self).__init__()

        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        self.config = RobertaConfig.from_pretrained("roberta-base")
        self.config.output_hidden_states = True
        self.roberta_layers = RobertaModel.from_pretrained("roberta-base", config = self.config)
        self.hidden_size = self.roberta_layers.config.hidden_size

        self.LSTM = torch.nn.LSTM(self.hidden_size, 384, num_layers, batch_first=True, bidirectional=bidirectional)
        self.out = nn.Linear(768, output_dim)

    def forward(self, text, mask, token_type_ids):
        #text = [batch size, sent len]
        # seq_layers, pooled_output = self.bert(text)
        embedded = self.roberta_layers(text, mask, token_type_ids)[0]

        # embedded = embedded[1]
        lstm_output, (last_hidden, _) = self.LSTM(embedded)
        #embedded = [batch size, emb dim]

        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=-1)
        output_hidden = F.dropout(output_hidden,0.2)
        
        # output_hidden = F.dropout(embedded, 0.2)
        output = self.out(output_hidden)

        #output = [batch size, out dim]
        return output


roberta =  RobertaModel.from_pretrained("roberta-base")       
model = BERTSentiment(roberta, 2).to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaMod

In [ ]:
# print(model(tokens.input_ids, tokens.attention_mask))
input_ids = tokens_pt.input_ids.to(device)
mask = tokens_pt.attention_mask.to(device)
print(model(input_ids, mask))

In [ ]:
SEED = 97
sarc_path = '/home/tegzes/Desktop/FL-Detection-Experiments/datamodule/isarcasm2022.csv'
BATCH_SIZE_TRAIN = 1
BATCH_SIZE_TEST = 1
MAX_LEN = 256
HIDDEN_DIM = 64
OUTPUT_DIM = 1
EMBEDDING_LENGTH = 300
N_LAYERS = 2
LEARNING_RATE = 1e-5
BIDIRECTIONAL = False
DROPOUT = 0.25
N_EPOCHS = 3
utils.seed_everything(SEED)


In [ ]:
train_iterator, valid_iterator, test_iterator = data.roberta_data_loader(sarc_path, BATCH_SIZE_TRAIN, BATCH_SIZE_TEST, True, 0, MAX_LEN, roberta_tokenizer, SEED)


In [158]:
# for batch_idx, batch in tqdm(enumerate(train_iterator, 0)):
batch = next(iter(train_iterator))
        
ids = batch['ids'].to(DEVICE, dtype = torch.long)
mask = batch['mask'].to(DEVICE, dtype = torch.long)
token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
targets = batch['targets'].to(DEVICE, dtype = torch.long)

# print(ids)
# print(mask)
# print(token_type_ids)
print(targets)
outputs = model(ids, mask, token_type_ids)
print(outputs)
_, predictions = torch.max(outputs.data, dim = 1)
print(predictions)
loss = cross_entropy_loss(outputs, targets)
print(loss)

tensor([0])
tensor([[ 0.0622, -0.0001]], grad_fn=<AddmmBackward0>)
tensor([0])
tensor(0.6624, grad_fn=<NllLossBackward0>)


In [107]:
cross_entropy_loss = nn.CrossEntropyLoss()
bce_loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct



In [ ]:
# task = Task.init(project_name="FL Detection", task_name="Training")

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    no_of_iterations = 0
    no_of_examples = 0
    
    # using torch metrics to calculate the metrics
    metric_acc = torchmetrics.Accuracy().to(torch.device("cuda", 0))
    metric_f1 = torchmetrics.F1(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_f1_micro = torchmetrics.F1(num_classes = 2).to(torch.device("cuda", 0))
    metric_f1_macro = torchmetrics.F1(num_classes = 2, average='macro').to(torch.device("cuda", 0))
    metric_precision = torchmetrics.Precision(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_recall = torchmetrics.Recall(num_classes = 2, average="none").to(torch.device("cuda", 0))
  
    
    model.train()

    for batch_idx, batch in tqdm(enumerate(iterator, 0)):
        
        ids = batch['ids'].to(DEVICE, dtype = torch.long)
        mask = batch['mask'].to(DEVICE, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
        targets = batch['targets'].to(DEVICE, dtype = torch.long)
        # tweet_lens = batch['tweet_len']

        outputs = model(ids, mask, token_type_ids)#, tweet_lens)#.to('cpu'))
        # outputs = model(ids, mask, token_type_ids)

        # targets = targets.unsqueeze(1) # for BCEWithLogitsLoss criterion
        loss = criterion(outputs, targets)
        epoch_loss += loss.item()

        _, predictions = torch.max(outputs.data, dim = 1)
        acc = calcuate_accuracy(predictions, targets)

        # loss = criterion(predictions, targets)
        # epoch_loss += loss.item()

        no_of_iterations += 1
        no_of_examples += targets.size(0)
                
        metric_acc.update(predictions, targets)
        metric_f1.update(outputs, targets)
        metric_f1_micro.update(outputs, targets)
        metric_f1_macro.update(outputs, targets)
        metric_precision.update(predictions, targets)
        metric_recall.update(predictions, targets)

        optimizer.zero_grad()
        loss.backward()
        # for GPU
        optimizer.step()

        # Logger.current_logger().report_scalar(
        #     "train", "loss", iteration = (epoch * len(iterator) + batch_idx), value = loss.item())
#----
        # print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(batch['ids']), len(iterator),
        #         100. * batch_idx / len(iterator), loss.item()))
        

    epoch_loss = epoch_loss/no_of_iterations
    epoch_acc = (acc*100)/no_of_examples
        
    acc_torch = metric_acc.compute()
    print(f"Training Accuracy: {acc_torch}")
    
    f1 = metric_f1.compute()
    print(f"Training F1: {f1}")
    
    f1_micro = metric_f1_micro.compute()
    print(f"Training F1 Micro: {f1_micro}")

    f1_macro = metric_f1_macro.compute()
    print(f"Training F1 Macro: {f1_macro}")
 
    precision = metric_precision.compute()
    print(f"Training Precision: {precision}")

    recall = metric_recall.compute()
    print(f"Training Recall: {recall}")
    
    print(f"Training Loss Epoch: {epoch_loss}")
  
    # Logger.current_logger().report_scalar(
    #     "train", "accuracy", iteration=epoch, value=acc_torch)

    
    metric_acc.reset()
    metric_f1.reset()
    metric_f1_micro.reset()
    metric_f1_macro.reset()
    metric_precision.reset()
    metric_recall.reset()
    
    return epoch_loss, epoch_acc

# evaluation routine
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    no_of_iterations = 0
    no_of_examples = 0    
    
   # torch metrics
    metric_acc = torchmetrics.Accuracy().to(torch.device("cuda", 0))
    metric_f1 = torchmetrics.F1(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_f1_micro = torchmetrics.F1(num_classes = 2).to(torch.device("cuda", 0))
    metric_f1_macro = torchmetrics.F1(num_classes = 2, average='macro').to(torch.device("cuda", 0))
    metric_precision = torchmetrics.Precision(num_classes = 2, average="none").to(torch.device("cuda", 0))
    metric_recall = torchmetrics.Recall(num_classes = 2, average="none").to(torch.device("cuda", 0))
  
    model.eval()
    
    with torch.no_grad():
    
        for _, batch in tqdm(enumerate(iterator, 0)):

            ids = batch['ids'].to(DEVICE, dtype = torch.long)
            mask = batch['mask'].to(DEVICE, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(DEVICE, dtype = torch.long)
            targets = batch['targets'].to(DEVICE, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids)
        
            _, predictions = torch.max(outputs.data, dim = 1)

            loss = criterion(outputs, targets)
            
            acc = calcuate_accuracy(predictions, targets)

            epoch_loss += loss.item()
    
            metric_acc.update(predictions, targets)
            metric_f1.update(outputs, targets)
            metric_f1_micro.update(outputs, targets)
            metric_f1_macro.update(outputs, targets)
            metric_precision.update(predictions, targets)
            metric_recall.update(predictions, targets)

            no_of_iterations += 1
            no_of_examples += targets.size(0)
    
    epoch_loss = epoch_loss/no_of_iterations
    epoch_acc = (acc*100)/no_of_iterations #no_of_examples

    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #     epoch_loss, acc, len(iterator),
    #     100. * acc / len(iterator)))

    acc_torch = metric_acc.compute()
    print(f"Validation Accuracy: {acc_torch}")
    
    f1 = metric_f1.compute()
    print(f"Validation F1 Validation: {f1}")
    
    f1_micro = metric_f1_micro.compute()
    print(f"Validation F1 Micro: {f1_micro}")

    f1_macro = metric_f1_macro.compute()
    print(f"Validation F1 Macro: {f1_macro}")
 
    precision = metric_precision.compute()
    print(f"Validation Precision: {precision}")

    recall = metric_recall.compute()
    print(f"Validation Recall: {recall}")
    
    print(f"Validation Loss Epoch: {epoch_loss}")

    # clear ml
    # Logger.current_logger().report_scalar(
    #     "test", "loss", iteration=epoch, value=epoch_loss)
    # Logger.current_logger().report_scalar(
    #     "test", "accuracy", iteration=epoch, value=acc_torch)

    metric_acc.reset()
    metric_f1.reset()
    metric_f1_micro.reset()
    metric_f1_macro.reset()
    metric_precision.reset()
    metric_recall.reset()
             
    return epoch_loss, epoch_acc


# experiment loop
for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, cross_entropy_loss)
    valid_loss, valid_acc = evaluate(model, valid_iterator, cross_entropy_loss)
        
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


test_loss, test_acc = evaluate(model, test_iterator, cross_entropy_loss)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

# task.close()